# Sub question 4a

*Which four countries have the highest electric car and truck sales relative to Gross Domestic Product (GDP), and how have sales trends in these countries developed between 2010 and 2022?*

Electric car/GDP sales per year

In [10]:
# Import helpfull packages

import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.io as pio
pio.renderers.default = 'plotly_mimetype+notebook'

In [11]:
# Import data from RQ2a
# Car data

df_cars = pd.read_csv('IEA-EV-dataEV salesHistoricalCars.csv')                                                      # Read csv file

df_cars_sales = df_cars[(df_cars['parameter'] == 'EV sales')]                                                       # Only look at EV sales values
df_cars_ev = df_cars_sales.drop(df_cars_sales[df_cars_sales['powertrain'] == 'FCEV'].index)                         # Drop the Fuel cell values
df_cars_group = df_cars_ev.groupby(['region', 'year'])['value'].sum()   	                                        # Group the dataframe by region and year to sum the values of the fully electric and hybrid for each year in each region
df_cars_values = df_cars_group.to_frame(name = 'value').reset_index()                                               # Make a dataframe of the groupby

df_cars_data = df_cars_values.drop(df_cars_values[(df_cars_values['region'] == 'EU27') |                            # Drop EU27, Europe, Rest of the world and World from the region values
                                                  (df_cars_values['region'] == 'Europe') | 
                                                  (df_cars_values['region'] == 'Rest of the world') |
                                                  (df_cars_values['region'] == 'World')].index)

years = pd.Series(range(2010, 2023))                                                                                # Years from 2010 to 2022
regions = df_cars_data['region'].unique()                                                                           # Unique regions in your data

df_full = pd.MultiIndex.from_product([years, regions], names=['year', 'region']).to_frame(index=False)              # Make new dataframe
df_cars_data_filled = pd.merge(df_full, df_cars_data, on=['year', 'region'], how='left')                            # Merge the complete grid with your original dataset
df_cars_data_filled['value'] = df_cars_data_filled['value'].fillna(0) 

# Truck data

df_trucks = pd.read_csv('IEA-EV-dataEV salesHistoricalTrucks.csv')                                                      # Read csv file

df_trucks_sales = df_trucks[(df_trucks['parameter'] == 'EV sales')]                                                     # Only look at EV sales values
df_trucks_ev = df_trucks_sales.drop(df_trucks_sales[df_trucks_sales['powertrain'] == 'FCEV'].index)                     # Drop the Fuel cell values
df_trucks_group = df_trucks_ev.groupby(['region', 'year'])['value'].sum()                                               # Group the dataframe by region and year to sum the values of the fully electric and hybrid for each year in each region
df_trucks_values = df_trucks_group.to_frame(name = 'value').reset_index()                                               # Make a dataframe of the groupby
df_trucks_data = df_trucks_values.drop(df_trucks_values[(df_trucks_values['region'] == 'EU27') |                        # Drop EU27, Europe, Rest of the world and World from the region values
                                                  (df_trucks_values['region'] == 'Europe') |
                                                  (df_trucks_values['region'] == 'Rest of the world') | 
                                                  (df_trucks_values['region'] == 'World')].index)
                                          

regions_trucks = df_trucks_data['region'].unique()                                                                      # Unique regions in your data
df_full_trucks = pd.MultiIndex.from_product([years, regions_trucks], names=['year', 'region']).to_frame(index=False)    # Make new dataframe
df_trucks_data_filled = pd.merge(df_full_trucks, df_trucks_data, on=['year', 'region'], how='left')                     # Merge the complete grid with your original dataset
df_trucks_data_filled['value'] = df_trucks_data_filled['value'].fillna(0)   

In [12]:

# Import data GDP

df_gdp = pd.read_csv('national-gdp-constant-usd-wb.csv')                                                        # Import csv file: GDP

# Make a usable dataframe for GDP use

df_gdp_years = df_gdp.drop(df_gdp[df_gdp['Year'] < 2010].index)                                                 # Remove all years below 2010
df_new_gdp = df_gdp_years.copy()                                                                                # Make a copy of the area_years dataframe
df_new_gdp['Entity'] = df_new_gdp['Entity'].replace({'United States': 'USA', 'Czechia': 'Czech Republic',       # Replace names of countries that are different in both csv files to make them equal
                                                       'South Korea': 'Korea', 'Turkey': 'Turkiye'})      



Car/GDP sales per year

In [ ]:
# Car/GDP per year

car_countries = df_cars_data['region'].unique()                                 # Select all unique countries in dataframe
df_gdp_y_car = df_new_gdp[df_new_gdp['Entity'].isin(car_countries)]             # Compare the gdp dataframe with the car countries dataframe

df_cars_gdp_merged = pd.merge(df_cars_data_filled, df_gdp_y_car, left_on=['region', 'year'], right_on=['Entity', 'Year'])                       # Merge new made dataframe with dataframe used in Subquestion 2
df_cars_gdp_merged['cars_per_gdp'] = df_cars_gdp_merged['value'] / df_cars_gdp_merged['GDP (constant 2015 US$)']           # Perform the division and create a new column for the result

df_cars_gdp_merged_sorted = df_cars_gdp_merged.sort_values(by=['year', 'cars_per_gdp'], ascending=[True, True])                                      # Sort the dataframe to get an ascending figure                                        

figure_cars_gdp = px.histogram(df_cars_gdp_merged_sorted, x='cars_per_gdp', y='region',                                                   # Plot figure
                           animation_frame='year', animation_group='region',
                           color='region', range_x=[0,0.0000006], title="Electric car sales per GDP per year",
                           text_auto='0.2s')
figure_cars_gdp.update_layout(yaxis={'categoryorder': 'total ascending'}, 
                                     xaxis_title = "Sales per GDP per year", 
                                     yaxis_title = 'Country', height=1000)
figure_cars_gdp.update_traces(textposition='outside')
figure_cars_gdp.show()


In [ ]:
# Percentage of electric car sales per GDP per year

total_sales_cars_per_year_gdp = df_cars_gdp_merged.groupby('year')['cars_per_gdp'].sum().reset_index()                # Calculate the total electric car sales for each year
total_sales_cars_per_year_gdp.columns = ['year', 'total_cars_per_gdp']                                            # Only look at year and total car sales per year to make %
df_cars_total_per_year_gdp = pd.merge(df_cars_gdp_merged, total_sales_cars_per_year_gdp, on='year')                   # Merge the total sales back into the original dataframe

df_cars_total_per_year_gdp['percentage_of_total_sales'] = (df_cars_total_per_year_gdp['cars_per_gdp'] / df_cars_total_per_year_gdp['total_cars_per_gdp']) * 100              # Calculate the percentage of each country's sales relative to the total for that year

figure_cars_i_perc = px.histogram(df_cars_total_per_year_gdp, x='percentage_of_total_sales', y='region',                                              # Plot figure
                           animation_frame='year', animation_group='region',
                           color='region', range_x=[0,110], title="Percentage of electric car sales per GDP per year", 
                           labels = {'region':'Countries'})

figure_cars_i_perc.update_layout(yaxis={'categoryorder': 'total ascending'}, height=1000, xaxis_title = "Percentage of sales per GDP per year", yaxis_title = 'Country')
figure_cars_i_perc.update_traces(textposition='outside', texttemplate='%{x:.2f}%')
figure_cars_i_perc.show()


In [ ]:
# Sum of electric car sales per gdp per year

df_car_sum_values_by_year_gdp = df_cars_gdp_merged.sort_values(by=['region', 'year'])                                                           # Sort values
df_car_sum_values_by_year_gdp['cumulative_sales_gdp'] = df_car_sum_values_by_year_gdp.groupby('region')['cars_per_gdp'].cumsum()                # Make new column with cumulative car/gdp sales per year

df_cars_sum_sorted_gdp = df_car_sum_values_by_year_gdp.sort_values(by=['year', 'cumulative_sales_gdp'], ascending=[True, True])                 # Sort dataframe by values to make a ascending figure

figure_cars_gdp_sum = px.histogram(df_cars_sum_sorted_gdp, x='cumulative_sales_gdp', y='region',                                                # Plot figure
                           animation_frame='year', animation_group='region',
                           color='region', range_x=[0,0.000002],
                           text_auto='0.2s',title="Sum of electric car sales per GDP per year")

figure_cars_gdp_sum.update_layout(yaxis={'categoryorder': 'total ascending'}, 
                                height=1000, xaxis_title = "Sum sales per GDP per year", 
                                yaxis_title = 'Country')
figure_cars_gdp_sum.update_traces(textposition='outside')
figure_cars_gdp_sum.show()

Electric Truck/GDP sales per year

In [ ]:
# Truck/GDP per year

trucks_countries = df_trucks_data['region'].unique()                                        # Select all unique countries in dataframe
df_gdp_y_trucks = df_new_gdp[df_new_gdp['Entity'].isin(trucks_countries)]                   # Compare the gdp dataframe with the truck countries dataframe

df_trucks_gdp_merged = pd.merge(df_trucks_data_filled, df_gdp_y_trucks, left_on=['region', 'year'], right_on=['Entity', 'Year'])                        # Merge new made dataframe with dataframe used in Subquestion 2
df_trucks_gdp_merged['trucks_per_gdp'] = df_trucks_gdp_merged['value'] / df_trucks_gdp_merged['GDP (constant 2015 US$)']                                # Perform the division and create a new column for the result

df_trucks_merged_sorted = df_trucks_gdp_merged.sort_values(by=['year', 'trucks_per_gdp'], ascending=[True, True])                                       # Sort the dataframe to get an ascending figure

figure_trucks_gdp = px.histogram(df_trucks_merged_sorted, x='trucks_per_gdp', y='region',                                                     # Plot figure
                           animation_frame='year', animation_group='region',
                           color='region', range_x=[0,0.0000000025],
                           text_auto='0.2s',title="Electric truck sales per GDP per year")

figure_trucks_gdp.update_layout(yaxis={'categoryorder': 'total ascending'}, 
                                       xaxis_title = "Sales per GDP per year", 
                                       yaxis_title = 'Country', height = 500)
figure_trucks_gdp.update_traces(textposition='outside')
figure_trucks_gdp.show()

In [ ]:
# Percentage of electric truck sales per GDP per year

total_sales_trucks_per_year_gdp = df_trucks_gdp_merged.groupby('year')['trucks_per_gdp'].sum().reset_index()                                          # Calculate the total electric truck sales for each year
total_sales_trucks_per_year_gdp.columns = ['year', 'total_trucks_per_gdp']                                                                        # Only look at year and total truck sales per year to make %
df_trucks_total_per_year_gdp = pd.merge(df_trucks_gdp_merged, total_sales_trucks_per_year_gdp, on='year')                                             # Merge the total sales back into the original dataframe

df_trucks_total_per_year_gdp['percentage_of_total_sales'] = (df_trucks_total_per_year_gdp['trucks_per_gdp'] / df_trucks_total_per_year_gdp['total_trucks_per_gdp']) * 100            # Calculate the percentage of each country's sales relative to the total for that year

figure_trucks_gdp_perc = px.histogram(df_trucks_total_per_year_gdp, x='percentage_of_total_sales', y='region',                                              # Plot figure
                           animation_frame='year', animation_group='region',
                           color='region', range_x=[0,110], title="Percentage of electric car sales per GDP per year", 
                           labels = {'region':'Countries'})

figure_trucks_gdp_perc.update_layout(yaxis={'categoryorder': 'total ascending'}, height=500, xaxis_title = "Percentage of sales per GDP per year", yaxis_title = 'Country')
figure_trucks_gdp_perc.update_traces(textposition='outside', texttemplate='%{x:.2f}%')
figure_trucks_gdp_perc.show()

In [ ]:
# Sum of electric car sales per GDP per year

df_trucks_sum_values_by_year_gdp = df_trucks_gdp_merged.sort_values(by=['region', 'year'])                                                                                       # Sort values
df_trucks_sum_values_by_year_gdp['cumulative_sales_gdp'] = df_trucks_sum_values_by_year_gdp.groupby('region')['trucks_per_gdp'].cumsum()                # Make new column with cumulative truck/gdp sales per year

df_trucks_sum_sorted_gdp = df_trucks_sum_values_by_year_gdp.sort_values(by=['year', 'cumulative_sales_gdp'], ascending=[True, True])                           # Sort dataframe by values to make a ascending figure

figure_trucks_gdp_sum = px.histogram(df_trucks_sum_sorted_gdp, x='cumulative_sales_gdp', y='region',                                              # Plot figure
                           animation_frame='year', animation_group='region',
                           color='region', range_x=[0,0.000025],
                           text_auto='0.2s',title="Sum of electric truck sales per GDP per year")

figure_trucks_gdp_sum.update_layout(yaxis={'categoryorder': 'total ascending'}, 
                            height=500, xaxis_title = "Sum sales per GDP per year", 
                            yaxis_title = 'Country',)
figure_trucks_gdp_sum.update_traces(textposition='outside')
figure_trucks_gdp_sum.show()